<a href="https://colab.research.google.com/github/aarondelgiudice/thinkful_data_bootcamp/blob/master/unit_4/lesson_3/433_Multi_Layer_Perception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Layer Perception
Unit 4 / Lesson 3 / Project 3

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.
Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [2]:
# load data
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [3]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


In [4]:
# Get data types
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [5]:
# Convert DateAcquired column to datetime
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
# Add a feature for year work was acquired
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [0]:
# Perform some addtional miscellaneous cleaning
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.


In [0]:
# define data and target
# get dummy variables

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

y = artworks.Department

In [8]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
mlp.score(X, y)

In [0]:
y.value_counts()/len(y)

In [0]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, y, cv=5)

Model seems to overfit.
This is a feature of neural networks that aren't given enough data for the number of features present. 

Note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

We included one parameter: hidden layer size.
This tells us how many and how big to make our layers.
Pass in a tuple that specifies each layer's size.
Our network, `(1000, )`, is 1000 neurons wide and one layer. `(100, 4, )` would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence.
It's generally less than the number of input variables you have.

You can also set an alpha.
Neural networks like this use a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section.
Alpha scales that penalty.

Lastly, we'll discuss the activation function.
The activation function determines whether the output from an individual perceptron is binary or continuous.
By default this is a 'relu', or 'rectified linear unit function' function.
In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative.
The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model.
It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation.
Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentaiton](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Experiment with different hidden layer structures and see how things vary.
Try this on a subset of the data to improve runtime.

Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [0]:
# set parameters
# model will use default parameters for the most part 
params = {'activation': 'relu'
          'solver': 'adam'
          'alpha': 0.0001
          'batch_size': 'auto'
          'learning_rate': 'constant'
          'learning_rate_init': 0.001
          # set warm_state=True
          # warm_state fits model on solution of previous call
          'warm_state': True
          # set early_stopping=True, keep tol=default
          # early_stopping will terminates training when
          # the validation score stops improving
          # requires solver='sgd' or 'adam'
          'early_stopping': True
          'tol': 1e-4
          'random_state': 42}

In [0]:
# change number of layers
layers = [(50, 100, 150), (1000, 1000, 1000, 1000, 1000)]

for layer in layers:
  # Instantiate and fit model
  model = MLPClassifier(**params, hidden_layer_size=layer).fit(X, y)
  # score
  model.score(X, y)
  # cross-validate
  cross_val_score(model, X, y, cv=5)

In [0]:
# change activation function
for layer in layers:
  # Instantiate and fit model
  # set activation function to sigmoid
  # called 'Logistic' in Scikit-Learn
  model = MLPClassifier(**params, hidden_layer_size=layer, 
                        activation='Logistic').fit(X, y)
  # score
  model.score(X, y)
  # cross-validate
  cross_val_score(model, X, y, cv=5)

In [0]:
# change learning rate
for layer in layers:
  # set learning_rate to 'adaptive'
  # Instantiate and fit model
  model = MLPClassifier(**params, hidden_layer_size=layer,
                       learning_rate='adaptive').fit(X, y)
  # score
  model.score(X, y)
  # cross-validate
  cross_val_score(model, X, y, cv=5)